<a href="https://colab.research.google.com/github/CanKeles5/ObjectDetection/blob/main/Detectron2_Faster_R_CNN_VisDrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 6.5MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=22b0f9475cf59706cea48d82038b3f95159fb3effdee0acdec927f3852d13b90
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.7.1+cu101 False
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import random
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from google.colab.patches import cv2_imshow
from detectron2.utils.visualizer import ColorMode

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [ ]:
!unzip /content/drive/MyDrive/VisDrone2019-DET-train.zip -d /content/

In [ ]:
!unzip /content/drive/MyDrive/VisDrone2019-DET-val.zip -d /content/

In [4]:
#/content/VisDrone2019-DET-train/annotations
def get_visdrone_dicts(img_path = "/content/VisDrone2019-DET-train/images", annot_path = "/content/VisDrone2019-DET-train/annotations" ):
  dataset_dicts = []
  for path, subdirs, files in os.walk(img_path):
   for filename in files:
     record = {}
     
     
     img_p = os.path.join(path, filename)
     anot_p = os.path.join(annot_path, filename[:-4] + '.txt')

     h, w = cv2.imread(img_p).shape[:2]

     record["file_name"] = img_p
     record["image_id"] = filename
     record["height"] = h
     record["width"] = w

     objs = []

     with open(anot_p) as fp:
       line = fp.readline()
       while line:
         line = line.replace("\n","") 
         vals = line.split (",")
         id = int(vals[5])
         
         if id==0 or id == 11:
           id=0
         elif id==1 or id==2:
           id=1
         else:
           id=2    
        
         b_left, b_top, b_width, b_height = list(map(float,vals[:4]))
         b_right, b_bottom = b_left + b_width, b_top - b_height
         
         obj = {
            "bbox": [b_left, b_top, b_width, b_height],
            "bbox_mode": BoxMode.XYWH_ABS,
            "segmentation": [],
            "category_id": id,
         }
         objs.append(obj)
         line = fp.readline()
     record["annotations"] = objs
     dataset_dicts.append(record)
  
  return dataset_dicts

In [34]:
DatasetCatalog.register("train_set",  lambda d=_: get_visdrone_dicts("/content/VisDrone2019-DET-train/images", "/content/VisDrone2019-DET-train/annotations"))
MetadataCatalog.get("train_set").set(thing_classes =["Others", "Person", "Vehicle"])
MetadataCatalog.get("train_set").thing_colors = [(0,255,0), (255,0,0), (0,0,255)]

DatasetCatalog.register("val_set",  lambda d=_: get_visdrone_dicts("/content/VisDrone2019-DET-val/images", "/content/VisDrone2019-DET-val/annotations"))
MetadataCatalog.get("val_set").set(thing_classes =["Others", "Person", "Vehicle"])
MetadataCatalog.get("val_set").thing_colors = [(0,255,0), (255,0,0), (0,0,255)]

In [40]:
train_metadata = MetadataCatalog.get("train_set")
val_metadata = MetadataCatalog.get("val_set")

In [41]:
val_metadata

Metadata(name='val_set', thing_classes=['Others', 'Person', 'Vehicle'], thing_colors=[(0, 255, 0), (255, 0, 0), (0, 0, 255)])

In [42]:
train_set_dicts = get_visdrone_dicts("/content/VisDrone2019-DET-train/images", "/content/VisDrone2019-DET-train/annotations")
val_set_dicts = get_visdrone_dicts("/content/VisDrone2019-DET-val/images", "/content/VisDrone2019-DET-val/annotations")

In [10]:
import json
from pathlib import Path
from PIL import Image as PILImage
import IPython
import numpy as np
from math import trunc
import base64
from io import BytesIO

In [11]:
#annot_path = "/content/drive/MyDrive/visdronecoco.json"
#img_path = "/content/VisDrone2019-DET-train/images"

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
)
cfg.DATASETS.TRAIN = ("train_set",)
cfg.DATASETS.TEST = ("val_set",)  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = (8000)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 12

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
!unzip /content/drive/MyDrive/VisDrone2019-DET-test-dev.zip -d /content/

In [ ]:
DatasetCatalog.register("datset10",  lambda d= _: get_visdrone_dicts("/content/images", "/content/annotations"))
MetadataCatalog.get("datset10").set(thing_classes =["Ignored", "Pedestrian", "People", "Bicycle", "Car", "Van", "Truck", "Tricycle", "awning-tricycle", "Bus", "Motor", "Others"])
MetadataCatalog.get("datset10").thing_colors = [(0,255,0), (255,0,0), (255,0,0), (0,0,255), (0,0,255), (0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,255,0)]

In [ ]:
balloon_metadata = MetadataCatalog.get("datset10")

In [ ]:
dataset_dicts = get_visdrone_dicts("/content/images", "/content/annotations")

In [ ]:
print(cfg.OUTPUT_DIR)

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/output/model_final.pth")  # path to the model we just trained
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4   # set a custom testing threshold
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.45
cfg.DATASETS.TEST = ( )
predictor = DefaultPredictor(cfg)

In [ ]:
for d in random.sample(dataset_dicts, 10):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=balloon_metadata, 
                   scale=1.5, 
                   instance_mode=ColorMode.SEGMENTATION   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])


In [ ]:
#import the COCO Evaluator to use the COCO Metrics
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

#Call the COCO Evaluator function and pass the Validation Dataset
evaluator = COCOEvaluator("datset9", cfg, False, output_dir="/output2/")
val_loader = build_detection_test_loader(cfg, "datset9")

#Use the created predicted model in the previous step
inference_on_dataset(predictor.model, val_loader, evaluator)